# **Tensorflow**
# key : tf.modulename.functionanae(parameters)
# key :  tensor <=>anydata  (yasari any data lai tensor(matrix) ra any tensor(matrix) lai data ma convert garxa  
[tensofflow for python ](https://www.tensorflow.org/)

# **STEPS** (TensorFlow ecosystem) [read step by step form tensorflow ecosystem](https://www.tensorflow.org/learn)
#C=create input piplines/Load & preprocess data
#R=Read Data
#U=update parameter /Build, train & reuse models
#D=Deploy model

# 1. Keras custom callback
# fit() ,evaluate(),predict() lai pass hunya keyword sabai keras call back hun 

In [28]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [0]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data()
x_train=x_train.reshape(60000,784).astype('float32')/255
x_test=x_test.reshape(10000,784).astype('float32')/255

In [30]:
print(x_test)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [0]:
def get_model():
  model=tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(1,activation='linear',input_dim=784))
  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.1),loss='mean_squared_error',metrics=['mae'])
  return model

In [0]:
from datetime import datetime
class madan(tf.keras.callbacks.Callback):
  def on_train_batch_begin(self,batch,logs=None):
    print('Training:Batch{}begin at {}'.format(batch,datetime.now().time()))

In [33]:
model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          epochs=1,
          steps_per_epoch=5,
          verbose=0,
          callbacks=[madan()])

Training:Batch0begin at 04:27:38.484607
Training:Batch1begin at 04:27:38.718066
Training:Batch2begin at 04:27:38.720823
Training:Batch3begin at 04:27:38.722752
Training:Batch4begin at 04:27:38.724418


# application of keras callback applications
# *Early stopping at minium loss
# yes main application of keras callback is jaba loss minium hunxa taba training afai stop hunuparyo ni ta.

In [0]:
class LossAndErrorPrintingCallback(tf.keras.callbacks.Callback):

  def on_train_batch_end(self, batch, logs=None):
    print('For batch {}, loss is {:7.2f}.'.format(batch, logs['loss']))

  def on_test_batch_end(self, batch, logs=None):
    print('For batch {}, loss is {:7.2f}.'.format(batch, logs['loss']))

  def on_epoch_end(self, epoch, logs=None):
    print('The average loss for epoch {} is {:7.2f} and mean absolute error is {:7.2f}.'.format(epoch, logs['loss'], logs['mae']))

In [0]:
import numpy as np
class EarlyStoppingMinLoss (tf.keras.callbacks.Callback):
  def __init__(self,patience=0):
    super(EarlyStoppingMinLoss,self).__init__()
    self.patience=patience
    self.best_weights=None
  def on_train_begin(self,logs=None):
    self.wait=0
    self.stopped_epoch=0
    self.best=np.Inf
  def on_epoch_end(self,epoch,logs=None):
    current=logs.get('loss')
    if np.less(current,self.best):
      self.best=current 
      self.wait=0
      self.best_weights=self.model.get_weights()
    else:
      self.wait+=1
      if self.wait>=self.patience:
        self.stopped_epoch=epoch
        self.model.stop_training=True
        print('Restoring Model weightd from the end of the best epoch ')
        self.model.set_weights(self.best_weights)
  def on_train_end (self,logs=None):
     if self.stopped_epoch>0:
        print('Epoch %05d :early stopping '% (self.stopped_epoch+1))


In [49]:
model=get_model()
_=model.fit(x_train,y_train,batch_size=64,steps_per_epoch=5,epochs=30,verbose=0,callbacks=[LossAndErrorPrintingCallback(),EarlyStoppingMinLoss()])

For batch 0, loss is   27.17.
For batch 1, loss is  840.05.
For batch 2, loss is   23.04.
For batch 3, loss is    8.43.
For batch 4, loss is    6.88.
The average loss for epoch 0 is  181.11 and mean absolute error is    7.96.
For batch 0, loss is   10.59.
For batch 1, loss is    6.07.
For batch 2, loss is    6.09.
For batch 3, loss is    4.58.
For batch 4, loss is    7.05.
The average loss for epoch 1 is    6.87 and mean absolute error is    2.07.
For batch 0, loss is    5.01.
For batch 1, loss is    4.93.
For batch 2, loss is    7.48.
For batch 3, loss is    5.41.
For batch 4, loss is    4.92.
The average loss for epoch 2 is    5.55 and mean absolute error is    1.90.
For batch 0, loss is    7.41.
For batch 1, loss is    7.01.
For batch 2, loss is    9.88.
For batch 3, loss is   22.44.
For batch 4, loss is   72.23.
The average loss for epoch 3 is   23.79 and mean absolute error is    3.84.
Restoring Model weightd from the end of the best epoch 
Epoch 00004 :early stopping 
